In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing Necessary Libraries

# Import Files

In [2]:
import shutil
import os
shutil.copyfile("/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/test_cloud.zip" , "test_cloud.zip")
shutil.copyfile("/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/submission_format.csv", "submission_format.csv")
shutil.copyfile("/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/3D_pointcloud_SimpleCNN_38for_cloud_size_32_64_64_acc_78.536_mcc_0.456.pth", "weight_3D.pth")

!unzip test_cloud.zip;

os.remove("test_cloud.zip")

path = "./test_cloud/"
submission_format_csv = "submission_format.csv"
checkpoint_model = "weight_3D.pth"

depth, height, width = 32, 64, 64   # dimension for converting point cloud to voxels

Archive:  test_cloud.zip
replace test_cloud/100032.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [3]:
checkpoint_model = 'weight_3D.pth'

In [4]:
import shutil
import os
import h5py
import csv

# importing the libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import time
import glob
import gc

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *
import torch.utils.data

%matplotlib inline


In [5]:
from torchvision.models.video import r3d_18
#from torchvision.models.video import r2plus1d_18
model = r3d_18(pretrained = False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.fc.out_features = 2

model.load_state_dict(torch.load(checkpoint_model))

<All keys matched successfully>

# Model

# Some Functions

In [6]:
def voxel_grid_from_cloud(cloud, out_depth=16, out_height=32, out_width=32):
    in_depth = np.max(cloud[:, 0])
    in_height = np.max(cloud[:, 1])
    in_width = np.max(cloud[:, 2])

    voxel_grid = np.zeros((out_depth, out_height, out_width), dtype=np.uint8)

    if in_depth >= out_depth:
        depth_transform_ratio = out_depth / (in_depth + 1)
        cloud[:, 0] = np.uint32(cloud[:, 0].astype(float) * depth_transform_ratio)
    if in_height >= out_height:
        height_transform_ratio = out_height / (in_height + 1)
        cloud[:, 1] = np.uint32(cloud[:, 1].astype(float) * height_transform_ratio)
    if in_width >= out_width:
        width_transform_ratio = out_width / (in_width + 1)
        cloud[:, 2] = np.uint32(cloud[:, 2].astype(float) * width_transform_ratio)

    voxel_grid[cloud[:, 0], cloud[:, 1], cloud[:, 2]] = 255

    return voxel_grid

In [7]:
def string2torch(test_y):
  df = pd.DataFrame(test_y, columns = ['Fname'])
  y = (df['Fname'].values)

  for i,filename in enumerate(y):
    seq_name = filename.split(".mp4")[0]
    y[i] = seq_name

  processed = np.array(y)
  processed = processed.astype(np.int)
  processed = torch.from_numpy(processed)

  return processed

In [8]:
def make_submission_file(filenames, y_pred):
  submit = []
  filenames = filenames.astype(int)
  for i in filenames:
    submit += [str(i)+'.mp4']

  submission_dict = {"filename": submit, "stalled": y_pred.astype(int)}
  submission_csv = pd.DataFrame(submission_dict)
  submission_csv.to_csv("submission3D.csv", index=False)
  

# Data Loading

In [9]:
with open(submission_format_csv, mode='r') as infile:
    reader = csv.reader(infile)
    test_list_csv = {rows[0]: rows[1] for rows in reader}
    infile.close()


files = list(test_list_csv.keys())
files.pop(0)
test_len = len(files)
print("Total data: " + str(test_len))

Total data: 14160


# Prediction and Submission file generation

In [10]:
y_pred = []
filenames = np.array([])


batch_size = 86

big_batch_size = 1000   # number of test data loaded at a time

for big_batch_no in range(math.ceil(test_len/big_batch_size)):

  this_batch_len = big_batch_size
  if (test_len - big_batch_size * big_batch_no) < big_batch_size:
    this_batch_len = test_len - big_batch_size * big_batch_no


  test_x = np.zeros((this_batch_len, 3, depth, height, width), dtype=np.uint8)
  test_yy = np.zeros(this_batch_len)


  # Load one big batch
  for i in range(this_batch_len):

    original_idx = i + big_batch_size * big_batch_no

    f = files[original_idx]
    original_name = f.replace(".mp4", "")
    h5_filename = path + original_name + ".h5"

    hf = h5py.File(h5_filename, 'r')
    cloud1 = hf['cloud1'][:]
    cloud2 = hf['cloud2'][:]
    cloud3 = hf['cloud3'][:]
    hf.close()


    test_x[i, 0, :, :, :] = voxel_grid_from_cloud(cloud1, out_depth=depth, out_height=height, out_width=width)
    test_x[i, 1, :, :, :] = voxel_grid_from_cloud(cloud2, out_depth=depth, out_height=height, out_width=width)
    test_x[i, 2, :, :, :] = voxel_grid_from_cloud(cloud3, out_depth=depth, out_height=height, out_width=width)
    test_yy[i] = int(original_name)


    print(original_idx, original_name, f, h5_filename)

  
  # Test one big batch
  test_y = torch.from_numpy(test_yy).int()
  ## change the data types from here
  test_x = torch.from_numpy(test_x).float()
  #test_y = torch.from_numpy(test_y)
  
  test = torch.utils.data.TensorDataset(test_x, test_y)
  test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False, num_workers=4)

  model.eval()
  for i,(images,labels) in tqdm(enumerate(test_loader)):
      #print(labels)
      filenames = np.append(filenames, labels)
      
      images = images.view(-1,3,depth,height,width)
      test = Variable(images.to(device), requires_grad=False)
      labels = Variable(labels.to(device), requires_grad=False)

      with torch.no_grad():
        # Forward propagation
        outputs = model(test)

        # Get predictions from the maximum value
        predicted = torch.max(outputs.data, 1)[1]
        #print(f"prediction size are {predicted.shape}")
        y_pred = np.append(y_pred, predicted.cpu().numpy())

#print(filenames)

make_submission_file(filenames, y_pred)


0 100032 100032.mp4 ./test_cloud/100032.h5
1 100037 100037.mp4 ./test_cloud/100037.h5
2 100139 100139.mp4 ./test_cloud/100139.h5
3 100182 100182.mp4 ./test_cloud/100182.h5
4 100214 100214.mp4 ./test_cloud/100214.h5
5 100215 100215.mp4 ./test_cloud/100215.h5
6 100222 100222.mp4 ./test_cloud/100222.h5
7 100231 100231.mp4 ./test_cloud/100231.h5
8 100239 100239.mp4 ./test_cloud/100239.h5
9 100294 100294.mp4 ./test_cloud/100294.h5
10 100308 100308.mp4 ./test_cloud/100308.h5
11 100331 100331.mp4 ./test_cloud/100331.h5
12 100347 100347.mp4 ./test_cloud/100347.h5
13 100357 100357.mp4 ./test_cloud/100357.h5
14 100380 100380.mp4 ./test_cloud/100380.h5
15 100510 100510.mp4 ./test_cloud/100510.h5
16 100520 100520.mp4 ./test_cloud/100520.h5
17 100659 100659.mp4 ./test_cloud/100659.h5
18 100660 100660.mp4 ./test_cloud/100660.h5
19 100708 100708.mp4 ./test_cloud/100708.h5
20 100798 100798.mp4 ./test_cloud/100798.h5
21 100822 100822.mp4 ./test_cloud/100822.h5
22 100825 100825.mp4 ./test_cloud/100825.h

12it [00:19,  1.64s/it]


1000 141592 141592.mp4 ./test_cloud/141592.h5
1001 141681 141681.mp4 ./test_cloud/141681.h5
1002 141707 141707.mp4 ./test_cloud/141707.h5
1003 141740 141740.mp4 ./test_cloud/141740.h5
1004 141770 141770.mp4 ./test_cloud/141770.h5
1005 141793 141793.mp4 ./test_cloud/141793.h5
1006 141851 141851.mp4 ./test_cloud/141851.h5
1007 141874 141874.mp4 ./test_cloud/141874.h5
1008 141955 141955.mp4 ./test_cloud/141955.h5
1009 142031 142031.mp4 ./test_cloud/142031.h5
1010 142267 142267.mp4 ./test_cloud/142267.h5
1011 142288 142288.mp4 ./test_cloud/142288.h5
1012 142314 142314.mp4 ./test_cloud/142314.h5
1013 142506 142506.mp4 ./test_cloud/142506.h5
1014 142593 142593.mp4 ./test_cloud/142593.h5
1015 142651 142651.mp4 ./test_cloud/142651.h5
1016 142691 142691.mp4 ./test_cloud/142691.h5
1017 142706 142706.mp4 ./test_cloud/142706.h5
1018 142753 142753.mp4 ./test_cloud/142753.h5
1019 142768 142768.mp4 ./test_cloud/142768.h5
1020 142817 142817.mp4 ./test_cloud/142817.h5
1021 143015 143015.mp4 ./test_clou

12it [00:20,  1.68s/it]


2000 181693 181693.mp4 ./test_cloud/181693.h5
2001 181709 181709.mp4 ./test_cloud/181709.h5
2002 181723 181723.mp4 ./test_cloud/181723.h5
2003 181807 181807.mp4 ./test_cloud/181807.h5
2004 181822 181822.mp4 ./test_cloud/181822.h5
2005 181834 181834.mp4 ./test_cloud/181834.h5
2006 181854 181854.mp4 ./test_cloud/181854.h5
2007 181874 181874.mp4 ./test_cloud/181874.h5
2008 181927 181927.mp4 ./test_cloud/181927.h5
2009 181983 181983.mp4 ./test_cloud/181983.h5
2010 182064 182064.mp4 ./test_cloud/182064.h5
2011 182162 182162.mp4 ./test_cloud/182162.h5
2012 182229 182229.mp4 ./test_cloud/182229.h5
2013 182239 182239.mp4 ./test_cloud/182239.h5
2014 182514 182514.mp4 ./test_cloud/182514.h5
2015 182544 182544.mp4 ./test_cloud/182544.h5
2016 182549 182549.mp4 ./test_cloud/182549.h5
2017 182557 182557.mp4 ./test_cloud/182557.h5
2018 182565 182565.mp4 ./test_cloud/182565.h5
2019 182602 182602.mp4 ./test_cloud/182602.h5
2020 182623 182623.mp4 ./test_cloud/182623.h5
2021 182663 182663.mp4 ./test_clou

...

13000 638471 638471.mp4 ./test_cloud/638471.h5
13001 638682 638682.mp4 ./test_cloud/638682.h5
13002 638702 638702.mp4 ./test_cloud/638702.h5
13003 638710 638710.mp4 ./test_cloud/638710.h5
13004 638750 638750.mp4 ./test_cloud/638750.h5
13005 638757 638757.mp4 ./test_cloud/638757.h5
13006 638778 638778.mp4 ./test_cloud/638778.h5
13007 638843 638843.mp4 ./test_cloud/638843.h5
13008 638886 638886.mp4 ./test_cloud/638886.h5
13009 638901 638901.mp4 ./test_cloud/638901.h5
13010 638966 638966.mp4 ./test_cloud/638966.h5
13011 639007 639007.mp4 ./test_cloud/639007.h5
13012 639159 639159.mp4 ./test_cloud/639159.h5
13013 639176 639176.mp4 ./test_cloud/639176.h5
13014 639198 639198.mp4 ./test_cloud/639198.h5
13015 639259 639259.mp4 ./test_cloud/639259.h5
13016 639316 639316.mp4 ./test_cloud/639316.h5
13017 639349 639349.mp4 ./test_cloud/639349.h5
13018 639361 639361.mp4 ./test_cloud/639361.h5
13019 639391 639391.mp4 ./test_cloud/639391.h5
13020 639419 639419.mp4 ./test_cloud/639419.h5
13021 639459 

12it [00:20,  1.71s/it]


14000 680992 680992.mp4 ./test_cloud/680992.h5
14001 681016 681016.mp4 ./test_cloud/681016.h5
14002 681093 681093.mp4 ./test_cloud/681093.h5
14003 681143 681143.mp4 ./test_cloud/681143.h5
14004 681229 681229.mp4 ./test_cloud/681229.h5
14005 681271 681271.mp4 ./test_cloud/681271.h5
14006 681308 681308.mp4 ./test_cloud/681308.h5
14007 681311 681311.mp4 ./test_cloud/681311.h5
14008 681325 681325.mp4 ./test_cloud/681325.h5
14009 681403 681403.mp4 ./test_cloud/681403.h5
14010 681498 681498.mp4 ./test_cloud/681498.h5
14011 681510 681510.mp4 ./test_cloud/681510.h5
14012 681596 681596.mp4 ./test_cloud/681596.h5
14013 681661 681661.mp4 ./test_cloud/681661.h5
14014 681776 681776.mp4 ./test_cloud/681776.h5
14015 681779 681779.mp4 ./test_cloud/681779.h5
14016 681896 681896.mp4 ./test_cloud/681896.h5
14017 681900 681900.mp4 ./test_cloud/681900.h5
14018 682046 682046.mp4 ./test_cloud/682046.h5
14019 682058 682058.mp4 ./test_cloud/682058.h5
14020 682098 682098.mp4 ./test_cloud/682098.h5
14021 682101 

2it [00:03,  1.75s/it]


'\nsubmission_dict = {"filename": filenames, "stalled": y_pred.astype(int)}\n\nsubmission_csv = pd.DataFrame(submission_dict)\n\nsubmission_csv.to_csv("submission_file.csv", index=False)\n'

In [13]:
filenames

array([100032, 100037, 100139, ..., 686906, 686958, 686995])